I was trying to update yt to the most recent version so that the new functions in setting yt fontsize could be used. The typical ``$HOME/anaconda/conda update yt `` did not work because anaconda has already updated to the latest version 3.2.1 of yt. The problem is that when you go into the ``anaconda/bin`` and try just running the executable ``./yt``, this error shows up : 

This error also shows up when I run ``./numba`` in the same directory. So really the problem is in the numpy installation. Looking at the traceback more carefully I realized that even though the yt installation is the anaconda version, it is still trying to call yt installation in my pip installation directory, or the default python directory on edison (from ``module load python``), and not the anaconda one

To fix this I needed to change my $PATH variable, which pointed to usr/bin/python or something thatś a non anaconda python executable. Then I also needed to change my $PYTHONPATH which tells the python interpreter where my python files lies:

In [1]:
import sys
sys.path

['',
 '//anaconda/lib/python27.zip',
 '//anaconda/lib/python2.7',
 '//anaconda/lib/python2.7/plat-linux2',
 '//anaconda/lib/python2.7/lib-tk',
 '//anaconda/lib/python2.7/lib-old',
 '//anaconda/lib/python2.7/lib-dynload',
 '//anaconda/lib/python2.7/site-packages',
 '//anaconda/lib/python2.7/site-packages/Sphinx-1.3.1-py2.7.egg',
 '//anaconda/lib/python2.7/site-packages/cryptography-0.9.1-py2.7-linux-x86_64.egg',
 '//anaconda/lib/python2.7/site-packages/rpy2-2.5.6-py2.7-linux-x86_64.egg',
 '//anaconda/lib/python2.7/site-packages/setuptools-17.1.1-py2.7.egg',
 '//anaconda/lib/python2.7/site-packages/singledispatch-3.4.0.3-py2.7.egg',
 '//anaconda/lib/python2.7/site-packages/IPython/extensions']

So all the messy stuff is removed:I removed the .local/python stuff directory (where pip installs its stuff if I used the user option -U)and also the default python stuff, I also took out mypythonpackage just to be on the safe end

Then after the ``$PATH`` and ``$PYTHONPATH`` are changed, I uninstalled anaconda, reinstalled it: 

It is important now to set the ``$PYTHONPATH=$HOME/anaconda/bin/`` __ONLY__!! so that python doesnt get confused about what numpy or what scipy to use when installing stuff.

(These PATH variables goes away when you relogin again, so it is easier if you put these PATH variable definitions in your .bashrc.ext or .ash_profile files )

In [ ]:
$HOME/anaconda/bin/conda install yt

Now if I do ./yt the thing runs sucessfully on command line, spitting back a bunch of options rather than the error, so tha's good '

But for some strange reason, the ipython notebook web interface doesnt really recognize the fact that we changed the anaconda python installation, which can be seen here : 

In [1]:
import sys
print(sys.version)
print sys.executable
print sys.exc_clear()

2.7.10 |Anaconda 2.2.0 (64-bit)| (default, May 28 2015, 17:02:03) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
//anaconda/bin/python
None


But the $HOME/anaconda/bin/python/ executable shows this newer version (note the new release date and version )

It was tricky because I tried removing stuff in .ipython/profile_default/ in order to wash out the ipython.nersc.gov 's memory of the prior version of anaconda. It didnt make a lot of sense because there is nothing there in the $HOME/anaconda/bin/python location now, so I have no idea what its actually being used to run that Anaconda 2.2. 

So it just keeps complaining yt can not be found because Ive deleted the paths that point to the bad installations 

In [3]:
import yt

ImportError: No module named yt

This is latter fixed by the solution provided by NERSC support last time I was trying to do this. The trick that since the ipthon.nersc.gov interface already has some anaconda stuff installed it is better to more the extra stuff that I installed (i.e yt) into a new location that ipython.nersc.gov find it 

where x is the version number 

Then I need to append the PATH into the ipython notebook everytime to 

In [4]:
sys.path.append('/global/homes/d/dorislee/mypackages')                                                                                          

Then import ..........nothing crashes !!! 

In [5]:
import yt

That sys path append statement is going to be put inside the ``$HOME/.ipython/profile_default/ipython_config.py`` which runs it everytime during startup 